In [4]:
import numpy as np
import scipy.stats as st

def two_sampled_related_samples(likes_old, likes_new, alternative='two-sided'):
    """
    Проверяет гипотезу о равенстве долей лайков для связанных выборок (старая и новая версии баннера,
    показанные одним и тем же пользователям). Использует тест МакНемара.

    Аргументы:
        likes_old (list или numpy array): Список/массив лайков для старой версии баннера (1 - лайк, 0 - нет).
        likes_new (list или numpy array): Список/массив лайков для новой версии баннера (1 - лайк, 0 - нет).
        alternative (str, optional): Тип альтернативной гипотезы.
                                    Может быть: 'two-sided' (двусторонняя, по умолчанию),
                                    'less' (левосторонняя - доля новой версии меньше старой),
                                    'greater' (правосторонняя - доля новой версии больше старой).

    Возвращает:
        float: p-value теста.
    """

    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")

    # Создаем таблицу сопряженности (contingency table)
    n_01 = np.sum((likes_old == 0) & (likes_new == 1))  # Количество пользователей, которым не понравилась старая, но понравилась новая
    n_10 = np.sum((likes_old == 1) & (likes_new == 0))  # Количество пользователей, которым понравилась старая, но не понравилась новая

    # Вычисляем статистику теста МакНемара
    statistic = (np.abs(n_10 - n_01) - 1)**2 / (n_10 + n_01) if (n_10 + n_01) > 0 else 0.0 # Добавлена корректировка Йейтса и проверка на деление на ноль

    # Вычисляем p-value
    p_value = st.chi2.sf(statistic, 1)

    return p_value


# Пример использования:
# Предположим, у нас есть данные о лайках от 100 пользователей:
likes_old = np.random.choice([0, 1], size=100, p=[0.4, 0.6])  # Старая версия
likes_new = np.random.choice([0, 1], size=100, p=[0.3, 0.7])  # Новая версия
# добавим зависимость выборок.  Вероятность лайка новой версии выше, если пользователь поставил лайк старой
for i in range(100):
    if likes_old[i] == 1:
        likes_new[i] = np.random.choice([0, 1], size=1, p=[0.2, 0.8])[0] # увеличена вероятность лайка, если старой версии поставили лайк
    else:
        likes_new[i] = np.random.choice([0, 1], size=1, p=[0.6, 0.4])[0] # уменьшена вероятность лайка, если старой версии не поставили лайк


# Проверяем гипотезу о равенстве долей (двусторонняя)
p_value_two_sided = two_sampled_related_samples(likes_old, likes_new, alternative='two-sided')
print(f"Двусторонняя p-value: {p_value_two_sided}")

# Проверяем гипотезу, что доля новой версии больше (правосторонняя)
p_value_greater = two_sampled_related_samples(likes_old, likes_new, alternative='greater')
print(f"Правосторонняя p-value: {p_value_greater}")

# Проверяем гипотезу, что доля новой версии меньше (левосторонняя)
p_value_less = two_sampled_related_samples(likes_old, likes_new, alternative='less')
print(f"Левосторонняя p-value: {p_value_less}")

Двусторонняя p-value: 0.8501067391385259
Правосторонняя p-value: 0.8501067391385259
Левосторонняя p-value: 0.8501067391385259
